In [2]:
from bs4 import BeautifulSoup
from bs4.element import NavigableString, Tag
import requests
import json
import re

url = 'https://libraryofmetzofitz.fandom.com/wiki/'







class ArchivesNethysScraper:
    def __init__(self, url, class_name):
        self.url = url
        self.href = f'{class_name}'
        self.page = requests.get(url + self.href)
        self.soup = BeautifulSoup(self.page.content, 'html.parser')
        self.table = self.soup.find("table")

    def table_rows(self):
        for b in self.soup.find_all('i')[1:]:
            print(b)
            yield b

    def clean_text(self, text):
        # Replace Unicode right single quotation mark with an apostrophe
        cleaned_text = [line.replace('\u2019', "'").replace('\u2018', "'").replace('\u201c', '"').replace('\u201d', '"').strip() for line in text if line.strip() and line.strip() != "."]
        return ' '.join(cleaned_text)
    
    def remove_parenthesis(self, input_string):
        pattern = r"\(.*?pg\..*?\)"
        result = re.sub(pattern, '', input_string)
        return result


    def get_class_info(self, table_row):
        output = {}
        # Find all h3 and b tags
        tags = self.soup.find_all(['i', 'em'])

        # Iterate over each tag
        for tag in tags:
            # Extract tag text and convert to lowercase as key
            key = tag.text.lower()
            # Extract all navigable strings until the next h3 or b tag
            value_tags = []
            next_sibling = tag.next_sibling

            while next_sibling and (not isinstance(next_sibling, Tag) or next_sibling.name not in ['i', 'em']):
                if hasattr(next_sibling, 'strings'):
                    # Join all strings to handle multiple lines
                    value_tags.append(' '.join(next_sibling.strings).strip())

                next_sibling = next_sibling.next_sibling

            # Clean the extracted value
            cleaned_value = self.clean_text(value_tags)
            cleaned_value = self.remove_parenthesis(cleaned_value)
            output[key] = cleaned_value

        return output
     



def get_class_info_json(url, class_name):
    scraper = ArchivesNethysScraper(url, class_name)
    class_info = scraper.get_class_info(next(scraper.table_rows(), None).string)
    filtered_info = {k: v for k, v in class_info.items() if v}
    return json.dumps(filtered_info, indent=2)

class_names = ["Zealot_Convictions"]

json_output = '{\n' + ',\n'.join([
    f'  "{class_name}": {get_class_info_json(url, class_name)}'
    for class_name in class_names
]) + '\n}'

print(json_output)

<i>Shielded Collective (Su)</i>
{
  "Zealot_Convictions": {
  "combat conviction": ": The zealot gains a\u00a0combat feat\u00a0he qualifies for as a bonus feat. After the zealot reaches 10th level, he can gain this conviction a second time.",
  "enduring zeal (su)": ": When the zealot activates his zeal class feature, he can spend up to 1 power point per zealot level to extend the duration of his zeal by a number of rounds equal to the power points spent.",
  "hearten the mindless (su)": ": Creatures in the zealot's collective that are mindless or immune to\u00a0mind-affecting\u00a0effects still benefit from his zeal class feature, gaining morale bonuses as if they were not mindless or immune to\u00a0mind-affecting\u00a0effects.",
  "improved zeal (su)": ": Whenever the zealot activates his zeal class feature, he can choose to pay 2 power points. If he does so, the intensity of his zeal increases by 1 for that use of his zeal class feature. A zealot must be at least 6th level before se

In [3]:
import json

def get_class_info_json(url, class_name):
    scraper = ArchivesNethysScraper(url, class_name)
    class_info = scraper.get_class_info(next(scraper.table_rows(), None).string)
    filtered_info = {k: v for k, v in class_info.items() if v}
    return filtered_info

class_names = ["Zealot_Convictions"]

# Create a dictionary to store class information
class_info_dict = {}
for class_name in class_names:
    class_info_dict[class_name] = get_class_info_json(url, class_name)

# Write the dictionary to a JSON file
with open('zealot.json', 'w') as json_file:
    json.dump(class_info_dict, json_file, indent=2)

<i>Shielded Collective (Su)</i>
